In [1]:
import os

In [2]:
%pwd

'/home/paladin/Downloads/Facial_Impression_Recognition_Calassification/research'

In [2]:
%pwd
os.chdir("../")

In [3]:
import os
import sys
import pandas as pd
from cnnClassifier.exception import CustomException
from cnnClassifier.logger import logging
from io import StringIO

import boto3
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories, get_size

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
   root_dir: Path
   s3_bucket: str
   s3_key: str
   s3_secret_key: str
   object_key: Path
   local_data_file: Path
    

In [5]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion   
        secret = self.secret.aws_credential

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            s3_bucket = secret.s3_bucket,
            s3_key = secret.s3_key,
            s3_secret_key= secret.s3_secret_key,
            object_key = secret.object_key,
            local_data_file=config.local_data_file

        )

        return data_ingestion_config

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.client = boto3.client('s3',
                      aws_access_key_id=self.config.s3_key,
                      aws_secret_access_key=self.config.s3_secret_key
                      )

    def dowload_file(self):
        if not os.path.exists(self.config.local_data_file):
            csv_obj = self.client.get_object(Bucket=self.config.s3_bucket, Key=self.config.object_key)
            body = csv_obj['Body']
            csv_string = body.read().decode('utf-8')
            df = pd.read_csv(StringIO(csv_string))
            df.to_csv(self.config.local_data_file, index=False, header=True)

            logging.info(f'{self.config.local_data_file} is downloaded!')
        else:
            logging.info(f"File alraedy exists of size : {get_size(Path(self.config.local_data_file))}")   

In [7]:
try:
    config = configurationManeger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.dowload_file()

except Exception as e:
    raise CustomException(e, sys)